In [1]:
import os

base = "madly_safe"

os.makedirs(os.path.join(base, "src"), exist_ok=True)
os.makedirs(os.path.join(base, "data"), exist_ok=True)
os.makedirs(os.path.join(base, "notebooks"), exist_ok=True)
os.makedirs(os.path.join(base, "models"), exist_ok=True)

# Crear README vacío
readme_path = os.path.join(base, "README.md")
if not os.path.exists(readme_path):
    with open(readme_path, "w", encoding="utf-8") as f:
        f.write("# MADly Safe\n")


In [3]:
import os

src_path = "src"  # estamos dentro de madly_safe, así que es solo "src"
files = ["__init__.py", "app.py", "etl.py", "model.py", "graphics.py"]

for fname in files:
    fpath = os.path.join(src_path, fname)
    if not os.path.exists(fpath):
        with open(fpath, "w", encoding="utf-8") as f:
            if fname == "__init__.py":
                f.write("# Permite importar el paquete src\n")
            else:
                f.write(f"# {fname} - se rellenará más adelante\n")



In [4]:
import os

req_path = "requirements.txt"

if not os.path.exists(req_path):
    with open(req_path, "w", encoding="utf-8") as f:
        f.write("dash\n")
        f.write("pandas\n")
        f.write("plotly\n")
        f.write("scikit-learn\n")

req_path, os.path.getsize(req_path)


('requirements.txt', 36)

In [5]:
!pip install dash plotly


In [11]:
import os
os.getcwd()


'C:\\Users\\cscpd\\madly_safe'

In [15]:
gitignore_contenido = """# Entornos virtuales
venv/
.env/

# Archivos de Python
__pycache__/
*.pyc

# Notebooks
.ipynb_checkpoints/

# Logs
*.log
"""

with open(".gitignore", "w", encoding="utf-8") as f:
    f.write(gitignore_contenido)

print("Archivo .gitignore creado en:", os.path.abspath(".gitignore"))


Archivo .gitignore creado en: C:\Users\cscpd\madly_safe\.gitignore


In [16]:
import os

print("Directorio actual:", os.getcwd())
print(".gitignore existe?:", os.path.exists(".gitignore"))

# Para verlo listado, aunque Jupyter no lo muestre en la interfaz:
print("Contenido del directorio:")
print(os.listdir("."))


Directorio actual: C:\Users\cscpd\madly_safe
.gitignore existe?: True
Contenido del directorio:
['.gitignore', '.ipynb_checkpoints', 'data', 'ejec.ipynb', 'models', 'notebooks', 'README.md', 'requirements.txt', 'src']


In [18]:
import os
os.system("explorer .")


1

In [27]:
import os, sys

print("CWD:", os.getcwd())

# Asegurarnos de tener la carpeta src en el path
src_path = os.path.abspath("src")
if src_path not in sys.path:
    sys.path.append(src_path)

import model
print("Usando model desde:", model.__file__)

# Probamos un escenario cualquiera
riesgo, alternativas = model.calcular_riesgo(
    "Pasajero",
    "Turismo",
    "25-34",
    "Hombre",
    "CENTRO",
    "Lunes",
    "Manana_media",
    "Despejado",
)
print("Riesgo:", riesgo)
print("Alternativas:", alternativas)


CWD: C:\Users\cscpd\madly_safe
Usando model desde: C:\Users\cscpd\madly_safe\src\model.py
Riesgo: 0.14298554734704166
Alternativas: [('18:00–21:59 (Opción A)', 0.12305855235437742), ('14:00–17:59 (Opción B)', 0.15726771733595518), ('06:00–09:59 (Opción C)', 0.17382453692386685)]


In [28]:
graphics_code = """
# graphics.py
\"\"\"Funciones que devuelven figuras de Plotly utilizadas en la app.\"\"\"

import plotly.graph_objects as go


def figura_franjas(riesgo_principal, alternativas):
    \"\"\"Construye un gráfico de barras con la franja seleccionada
    y las franjas alternativas.

    Parameters
    ----------
    riesgo_principal : float
        Probabilidad para la franja seleccionada (0–1).
    alternativas : list of (str, float)
        Lista de pares (nombre_franja_legible, riesgo).

    Returns
    -------
    fig : plotly.graph_objects.Figure
    \"\"\"
    if riesgo_principal is None or alternativas is None:
        return figura_vacia()

    nombres = [\"Franja seleccionada\"] + [alt[0] for alt in alternativas]
    valores = [riesgo_principal] + [alt[1] for alt in alternativas]
    porcentajes = [v * 100 for v in valores]

    fig = go.Figure()
    fig.add_bar(
        x=nombres,
        y=valores,
        text=[f\"{p:.1f} %\" for p in porcentajes],
        textposition=\"auto\",
    )
    fig.update_layout(
        title=\"Comparación de franjas horarias\",
        yaxis=dict(
            title=\"Probabilidad de lesión grave\",
            tickformat=\".0%\",
            range=[0, max(valores) * 1.2],
        ),
        xaxis_title=\"Franja\",
        bargap=0.3,
    )

    return fig


def figura_vacia():
    \"\"\"Figura vacía para cuando aún no hay escenario completo.\"\"\"
    fig = go.Figure()
    fig.update_layout(
        title=\"Franjas alternativas\",
        xaxis_title=\"Franja\",
        yaxis_title=\"Probabilidad de lesión grave\",
    )
    return fig
"""

with open("src/graphics.py", "w", encoding="utf-8") as f:
    f.write(graphics_code)

print("src/graphics.py reescrito.")


src/graphics.py reescrito.


In [29]:
app_code = """
from dash import Dash, html, dcc, Input, Output

from model import calcular_riesgo
from graphics import figura_franjas, figura_vacia

# Creamos la app Dash
app = Dash(__name__, title="MADly Safe · Riesgo de lesión grave en Madrid")
server = app.server

# ----- Opciones de los controles -----

TIPOS_PERSONA = [
    {"label": "Conductor", "value": "Conductor"},
    {"label": "Pasajero", "value": "Pasajero"},
    {"label": "Peatón", "value": "Peatón"},
]

TIPOS_VEHICULO = [
    {"label": "Turismo", "value": "Turismo"},
    {"label": "Motocicleta", "value": "Motocicleta"},
    {"label": "Furgoneta", "value": "Furgoneta"},
    {"label": "Bicicleta", "value": "Bicicleta"},
    {"label": "VMP / Patinete", "value": "VMP"},
    {"label": "Sin vehículo (peatón)", "value": "Sin_vehiculo"},
]

RANGOS_EDAD = [
    {"label": "Menor de 18 años", "value": "<18"},
    {"label": "18–24 años", "value": "18-24"},
    {"label": "25–34 años", "value": "25-34"},
    {"label": "35–44 años", "value": "35-44"},
    {"label": "45–54 años", "value": "45-54"},
    {"label": "55–64 años", "value": "55-64"},
    {"label": "65–74 años", "value": "65-74"},
    {"label": "75+ años", "value": "75+"},
]

SEXO_OPCIONES = [
    {"label": "Hombre", "value": "Hombre"},
    {"label": "Mujer", "value": "Mujer"},
    {"label": "Desconocido / Otro", "value": "Desconocido"},
]

DISTRITOS = [
    {"label": "Centro", "value": "CENTRO"},
    {"label": "Arganzuela", "value": "ARGANZUELA"},
    {"label": "Retiro", "value": "RETIRO"},
    {"label": "Salamanca", "value": "SALAMANCA"},
    {"label": "Chamartín", "value": "CHAMARTIN"},
    {"label": "Tetuán", "value": "TETUAN"},
    {"label": "Chamberí", "value": "CHAMBERI"},
]

DIAS_SEMANA = [
    {"label": "Lunes", "value": "Lunes"},
    {"label": "Martes", "value": "Martes"},
    {"label": "Miércoles", "value": "Miércoles"},
    {"label": "Jueves", "value": "Jueves"},
    {"label": "Viernes", "value": "Viernes"},
    {"label": "Sábado", "value": "Sábado"},
    {"label": "Domingo", "value": "Domingo"},
]

METEOROLOGIA = [
    {"label": "Despejado", "value": "Despejado"},
    {"label": "Nublado", "value": "Nublado"},
    {"label": "Lluvia débil", "value": "Lluvia debil"},
    {"label": "Lluvia intensa", "value": "Lluvia intensa"},
    {"label": "Se desconoce", "value": "Desconocido"},
]

FRANJAS_HORARIAS = [
    {"label": "00:00 – 05:59", "value": "Noche_madrugada"},
    {"label": "06:00 – 09:59", "value": "Manana_punta"},
    {"label": "10:00 – 13:59", "value": "Manana_media"},
    {"label": "14:00 – 17:59", "value": "Tarde"},
    {"label": "18:00 – 21:59", "value": "Tarde_punta"},
    {"label": "22:00 – 23:59", "value": "Noche"},
]

# ----- Layout de la app -----

app.layout = html.Div(
    children=[
        # Cabecera
        html.Div(
            children=[
                html.H1("MADly Safe", style={"marginBottom": "5px"}),
                html.H3(
                    "Recomendador de franjas más seguras según perfil y contexto en Madrid",
                    style={"fontWeight": "normal", "color": "#444"},
                ),
                html.P(
                    "Selecciona tu perfil y condiciones de desplazamiento. "
                    "La aplicación estima la probabilidad de lesión grave "
                    "(condicionada a que ocurra un accidente) y sugiere franjas alternativas.",
                    style={"maxWidth": "900px"},
                ),
            ],
            style={
                "textAlign": "left",
                "padding": "20px 40px 10px 40px",
                "backgroundColor": "#f8f9fa",
                "borderBottom": "1px solid #ddd",
            },
        ),

        # Cuerpo: formulario + resultados
        html.Div(
            children=[
                # Columna izquierda: formulario
                html.Div(
                    children=[
                        html.H4("1. Define tu escenario"),

                        html.Label("Tipo de persona"),
                        dcc.Dropdown(
                            id="input-tipo-persona",
                            options=TIPOS_PERSONA,
                            value="Conductor",
                        ),
                        html.Br(),

                        html.Label("Tipo de vehículo"),
                        dcc.Dropdown(
                            id="input-tipo-vehiculo",
                            options=TIPOS_VEHICULO,
                            value="Turismo",
                        ),
                        html.Br(),

                        html.Label("Rango de edad"),
                        dcc.Dropdown(
                            id="input-rango-edad",
                            options=RANGOS_EDAD,
                            value="25-34",
                        ),
                        html.Br(),

                        html.Label("Sexo"),
                        dcc.Dropdown(
                            id="input-sexo",
                            options=SEXO_OPCIONES,
                            value="Hombre",
                        ),
                        html.Br(),

                        html.Label("Distrito de Madrid"),
                        dcc.Dropdown(
                            id="input-distrito",
                            options=DISTRITOS,
                            value="CENTRO",
                        ),
                        html.Br(),

                        html.Label("Día de la semana"),
                        dcc.Dropdown(
                            id="input-dia",
                            options=DIAS_SEMANA,
                            value="Lunes",
                        ),
                        html.Br(),

                        html.Label("Franja horaria"),
                        dcc.Dropdown(
                            id="input-franja",
                            options=FRANJAS_HORARIAS,
                            value="Tarde_punta",
                        ),
                        html.Br(),

                        html.Label("Estado meteorológico"),
                        dcc.Dropdown(
                            id="input-meteo",
                            options=METEOROLOGIA,
                            value="Despejado",
                        ),
                    ],
                    style={
                        "display": "inline-block",
                        "verticalAlign": "top",
                        "width": "30%",
                        "padding": "20px 40px",
                        "boxSizing": "border-box",
                        "borderRight": "1px solid #eee",
                    },
                ),

                # Columna derecha: resultados
                html.Div(
                    children=[
                        html.H4("2. Riesgo estimado y franjas alternativas"),
                        html.Div(
                            id="card-riesgo",
                            style={
                                "padding": "15px 20px",
                                "borderRadius": "10px",
                                "backgroundColor": "#fff3cd",
                                "border": "1px solid #ffeeba",
                                "marginBottom": "20px",
                            },
                        ),
                        dcc.Graph(
                            id="grafico-franjas",
                            style={"height": "380px"},
                        ),
                        html.Div(
                            id="explicacion",
                            style={"marginTop": "15px", "color": "#555"},
                        ),
                    ],
                    style={
                        "display": "inline-block",
                        "verticalAlign": "top",
                        "width": "70%",
                        "padding": "20px 40px",
                        "boxSizing": "border-box",
                    },
                ),
            ]
        ),
    ]
)

# ----- Callback -----


@app.callback(
    Output("card-riesgo", "children"),
    Output("grafico-franjas", "figure"),
    Output("explicacion", "children"),
    Input("input-tipo-persona", "value"),
    Input("input-tipo-vehiculo", "value"),
    Input("input-rango-edad", "value"),
    Input("input-sexo", "value"),
    Input("input-distrito", "value"),
    Input("input-dia", "value"),
    Input("input-franja", "value"),
    Input("input-meteo", "value"),
)
def actualizar_salida(tipo_persona, tipo_vehiculo, rango_edad, sexo,
                      distrito, dia, franja, meteo):

    try:
        riesgo, alternativas = calcular_riesgo(
            tipo_persona, tipo_vehiculo, rango_edad, sexo,
            distrito, dia, franja, meteo
        )
    except Exception as e:
        card = html.Div(
            [
                html.Div(
                    "Se ha producido un error al calcular el riesgo.",
                    style={"fontWeight": "bold"},
                ),
                html.Div(
                    "Revisa los valores seleccionados o contacta con la persona responsable de la aplicación.",
                    style={"fontSize": "13px"},
                ),
            ]
        )
        figura = figura_vacia()
        explicacion = f"Detalle técnico del error (solo para depuración): {e}"
        return card, figura, explicacion

    if riesgo is None or alternativas is None:
        card = html.Div(
            "Completa los campos de la izquierda para ver la estimación de riesgo.",
            style={"fontWeight": "bold"},
        )
        figura = figura_vacia()
        explicacion = ""
        return card, figura, explicacion

    riesgo_pct = round(riesgo * 100, 2)
    card = html.Div(
        [
            html.Div("Escenario seleccionado", style={"fontSize": "14px", "color": "#777"}),
            html.Div(
                f"{riesgo_pct} %",
                style={"fontSize": "34px", "fontWeight": "bold"},
            ),
            html.Div(
                "Probabilidad estimada de lesión grave o fallecimiento "
                "condicionada a que ocurra un accidente.",
                style={"fontSize": "13px"},
            ),
            html.Div(
                "Modelo actual: Regresión Logística (class_weight='balanced').",
                style={"fontSize": "12px", "color": "#666", "marginTop": "4px"},
            ),
        ]
    )

    fig = figura_franjas(riesgo, alternativas)

    nombres_alternativas = [alt[0] for alt in alternativas]
    alternativas_texto = ", ".join(nombres_alternativas)

    explicacion = (
        "Con el perfil seleccionado, la franja actual presenta un riesgo aproximado "
        f"del {riesgo_pct} %. "
        "Las franjas alternativas sugeridas son: "
        f"{alternativas_texto}. "
        "En todos los casos se mantiene fijo el resto del escenario "
        "(perfil, distrito, día de la semana y meteorología). "
        "Esta estimación se basa en un modelo estadístico entrenado con datos históricos "
        "de la ciudad de Madrid y debe interpretarse solo con fines informativos."
    )

    return card, fig, explicacion


if __name__ == "__main__":
    app.run(debug=False, use_reloader=False)
"""

with open("src/app.py", "w", encoding="utf-8") as f:
    f.write(app_code)

print("src/app.py reescrito.")


src/app.py reescrito.


In [31]:
!python src/app.py


^C


In [32]:
import os

print("CWD:", os.getcwd())
print("Contenido de src/:")
print(os.listdir("src"))


CWD: C:\Users\cscpd\madly_safe
Contenido de src/:
['.ipynb_checkpoints', 'app.py', 'etl.py', 'graphics.py', 'model.py', '__init__.py', '__pycache__']


In [33]:
with open("src/app.py", "r", encoding="utf-8") as f:
    contenido = f.read()

print(contenido[:500])



from dash import Dash, html, dcc, Input, Output

from model import calcular_riesgo
from graphics import figura_franjas, figura_vacia

# Creamos la app Dash
app = Dash(__name__, title="MADly Safe · Riesgo de lesión grave en Madrid")
server = app.server

# ----- Opciones de los controles -----

TIPOS_PERSONA = [
    {"label": "Conductor", "value": "Conductor"},
    {"label": "Pasajero", "value": "Pasajero"},
    {"label": "Peatón", "value": "Peatón"},
]

TIPOS_VEHICULO = [
    {"label": "Turismo"


In [2]:
!python src/app.py


^C


In [3]:
!python app.py

Traceback (most recent call last):
  File "app.py", line 3, in <module>
    from src.app import app, server
  File "C:\Users\cscpd\madly_safe\src\app.py", line 4, in <module>
    from model import calcular_riesgo
ModuleNotFoundError: No module named 'model'


In [4]:
!python app.py


Traceback (most recent call last):
  File "app.py", line 3, in <module>
    from src.app import app, server
  File "C:\Users\cscpd\madly_safe\src\app.py", line 4, in <module>
    from model import calcular_riesgo
ModuleNotFoundError: No module named 'model'


In [5]:
!python app.py


^C
